In [142]:
from __future__ import absolute_import, division, print_function
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path
from random import shuffle
import numpy as np
from tensorflow.data import Dataset
AUTOTUNE = tf.data.experimental.AUTOTUNE

keras = tf.keras

In [143]:
IMG_SIZE = 160 # All images will be resized to 160x160
BATCH_SIZE = 32
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_path = 'data/PetImages'

In [174]:
def load_files(base_path, split=[8, 1, 1]):
    # normalize splits
    splits = np.array(split) / np.sum(np.array(split))
    
    # find labels - parent folder names
    labels = { k.name: v for (v, k) in enumerate(Path(base_path).glob('*/')) }
    
    # load all files along with idx label
    data = [(str(f.absolute()), labels[f.parent.name]) for f in Path(base_path).glob('**/*.jpg')]
    
    # shuffle data
    shuffle(data)
    
    # split data
    train_idx = int(len(data) * splits[0])
    eval_idx = int(len(data) * splits[1])
    
    return data[:train_idx], \
            data[train_idx:train_idx + eval_idx], \
            data[train_idx + eval_idx:], \
            labels

In [363]:
train, test, val, labels = load_files('data/PetImages')
labels

{'Cat': 0, 'Dog': 1}

In [381]:
def process_item(path, label):
    img_raw = tf.io.read_file(path)
    img_tensor = tf.image.decode_jpeg(img_raw)
    img_final = tf.image.resize(img_tensor, [IMG_SIZE, IMG_SIZE]) / 255
    
    return img_final, label        

In [389]:
train_data, train_labels = zip(*train)

train_ds = Dataset.zip((Dataset.from_tensor_slices(list(train_data)),
                        Dataset.from_tensor_slices(list(train_labels))))

train_ds = train_ds.map(map_func=process_item, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.apply(tf.data.experimental.ignore_errors())
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
train_ds = train_ds.repeat()

In [390]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(len(labels))
])

In [391]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [392]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_6 ( (None, 1280)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [394]:
history = model.fit(train_ds, epochs=5, steps_per_epoch=tf.math.ceil(len(train)/BATCH_SIZE).numpy())

Epoch 1/5


185/625 [=======>......................] - ETA: 36:28 - loss: 5.3588 - accuracy: 0.515 - ETA: 18:53 - loss: 5.1618 - accuracy: 0.531 - ETA: 13:01 - loss: 4.5794 - accuracy: 0.552 - ETA: 10:00 - loss: 4.1893 - accuracy: 0.574 - ETA: 8:13 - loss: 4.0466 - accuracy: 0.584 - ETA: 7:03 - loss: 3.9227 - accuracy: 0.58 - ETA: 6:11 - loss: 3.8990 - accuracy: 0.57 - ETA: 5:32 - loss: 3.8869 - accuracy: 0.57 - ETA: 5:03 - loss: 3.7300 - accuracy: 0.58 - ETA: 4:39 - loss: 3.6725 - accuracy: 0.59 - ETA: 4:19 - loss: 3.7126 - accuracy: 0.59 - ETA: 4:04 - loss: 3.7037 - accuracy: 0.60 - ETA: 3:50 - loss: 3.7038 - accuracy: 0.60 - ETA: 3:39 - loss: 3.7061 - accuracy: 0.59 - ETA: 3:29 - loss: 3.7712 - accuracy: 0.59 - ETA: 3:21 - loss: 3.7389 - accuracy: 0.59 - ETA: 3:14 - loss: 3.7436 - accuracy: 0.60 - ETA: 3:06 - loss: 3.7117 - accuracy: 0.60 - ETA: 2:59 - loss: 3.6897 - accuracy: 0.59 - ETA: 2:54 - loss: 3.6851 - accuracy: 0.59 - ETA: 2:49 - loss: 3.7530 - accuracy: 0.59 - ETA: 2:44 - loss: 3.7359

371/625 [================>.............] - ETA: 51s - loss: 2.8527 - accuracy: 0.653 - ETA: 51s - loss: 2.8462 - accuracy: 0.654 - ETA: 50s - loss: 2.8459 - accuracy: 0.654 - ETA: 50s - loss: 2.8421 - accuracy: 0.654 - ETA: 50s - loss: 2.8338 - accuracy: 0.654 - ETA: 50s - loss: 2.8284 - accuracy: 0.654 - ETA: 50s - loss: 2.8281 - accuracy: 0.654 - ETA: 50s - loss: 2.8262 - accuracy: 0.654 - ETA: 50s - loss: 2.8203 - accuracy: 0.654 - ETA: 50s - loss: 2.8129 - accuracy: 0.654 - ETA: 49s - loss: 2.8063 - accuracy: 0.655 - ETA: 49s - loss: 2.8072 - accuracy: 0.655 - ETA: 49s - loss: 2.8020 - accuracy: 0.655 - ETA: 49s - loss: 2.7978 - accuracy: 0.656 - ETA: 49s - loss: 2.7920 - accuracy: 0.656 - ETA: 49s - loss: 2.7851 - accuracy: 0.657 - ETA: 49s - loss: 2.7801 - accuracy: 0.657 - ETA: 48s - loss: 2.7770 - accuracy: 0.658 - ETA: 48s - loss: 2.7739 - accuracy: 0.658 - ETA: 48s - loss: 2.7741 - accuracy: 0.658 - ETA: 48s - loss: 2.7675 - accuracy: 0.659 - ETA: 48s - loss: 2.7652 - accurac

558/625 [=========================>....] - ETA: 26s - loss: 2.3856 - accuracy: 0.699 - ETA: 26s - loss: 2.3835 - accuracy: 0.699 - ETA: 26s - loss: 2.3805 - accuracy: 0.699 - ETA: 26s - loss: 2.3783 - accuracy: 0.699 - ETA: 26s - loss: 2.3771 - accuracy: 0.700 - ETA: 26s - loss: 2.3732 - accuracy: 0.700 - ETA: 26s - loss: 2.3706 - accuracy: 0.700 - ETA: 25s - loss: 2.3669 - accuracy: 0.701 - ETA: 25s - loss: 2.3649 - accuracy: 0.701 - ETA: 25s - loss: 2.3663 - accuracy: 0.701 - ETA: 25s - loss: 2.3659 - accuracy: 0.701 - ETA: 25s - loss: 2.3641 - accuracy: 0.701 - ETA: 25s - loss: 2.3629 - accuracy: 0.701 - ETA: 25s - loss: 2.3609 - accuracy: 0.702 - ETA: 25s - loss: 2.3586 - accuracy: 0.702 - ETA: 24s - loss: 2.3549 - accuracy: 0.702 - ETA: 24s - loss: 2.3516 - accuracy: 0.703 - ETA: 24s - loss: 2.3501 - accuracy: 0.703 - ETA: 24s - loss: 2.3512 - accuracy: 0.703 - ETA: 24s - loss: 2.3486 - accuracy: 0.703 - ETA: 24s - loss: 2.3482 - accuracy: 0.703 - ETA: 24s - loss: 2.3470 - accurac

625/625 [==============================] - ETA: 6s - loss: 2.0739 - accuracy: 0.73 - ETA: 6s - loss: 2.0732 - accuracy: 0.73 - ETA: 6s - loss: 2.0711 - accuracy: 0.73 - ETA: 6s - loss: 2.0702 - accuracy: 0.73 - ETA: 6s - loss: 2.0680 - accuracy: 0.73 - ETA: 6s - loss: 2.0665 - accuracy: 0.73 - ETA: 6s - loss: 2.0653 - accuracy: 0.73 - ETA: 5s - loss: 2.0634 - accuracy: 0.73 - ETA: 5s - loss: 2.0618 - accuracy: 0.73 - ETA: 5s - loss: 2.0590 - accuracy: 0.73 - ETA: 5s - loss: 2.0576 - accuracy: 0.73 - ETA: 5s - loss: 2.0560 - accuracy: 0.73 - ETA: 5s - loss: 2.0563 - accuracy: 0.73 - ETA: 5s - loss: 2.0544 - accuracy: 0.73 - ETA: 5s - loss: 2.0533 - accuracy: 0.73 - ETA: 5s - loss: 2.0520 - accuracy: 0.73 - ETA: 5s - loss: 2.0530 - accuracy: 0.73 - ETA: 4s - loss: 2.0513 - accuracy: 0.73 - ETA: 4s - loss: 2.0512 - accuracy: 0.73 - ETA: 4s - loss: 2.0502 - accuracy: 0.73 - ETA: 4s - loss: 2.0484 - accuracy: 0.73 - ETA: 4s - loss: 2.0477 - accuracy: 0.73 - ETA: 4s - loss: 2.0457 - accuracy

...

625/625 [==============================] - ETA: 7s - loss: 0.6647 - accuracy: 0.91 - ETA: 6s - loss: 0.6641 - accuracy: 0.91 - ETA: 6s - loss: 0.6639 - accuracy: 0.91 - ETA: 6s - loss: 0.6652 - accuracy: 0.91 - ETA: 6s - loss: 0.6641 - accuracy: 0.91 - ETA: 6s - loss: 0.6640 - accuracy: 0.91 - ETA: 6s - loss: 0.6657 - accuracy: 0.91 - ETA: 6s - loss: 0.6655 - accuracy: 0.91 - ETA: 6s - loss: 0.6645 - accuracy: 0.91 - ETA: 6s - loss: 0.6639 - accuracy: 0.91 - ETA: 5s - loss: 0.6632 - accuracy: 0.91 - ETA: 5s - loss: 0.6621 - accuracy: 0.91 - ETA: 5s - loss: 0.6610 - accuracy: 0.91 - ETA: 5s - loss: 0.6605 - accuracy: 0.91 - ETA: 5s - loss: 0.6620 - accuracy: 0.91 - ETA: 5s - loss: 0.6613 - accuracy: 0.91 - ETA: 5s - loss: 0.6616 - accuracy: 0.91 - ETA: 5s - loss: 0.6605 - accuracy: 0.91 - ETA: 5s - loss: 0.6605 - accuracy: 0.91 - ETA: 5s - loss: 0.6600 - accuracy: 0.91 - ETA: 4s - loss: 0.6589 - accuracy: 0.91 - ETA: 4s - loss: 0.6578 - accuracy: 0.91 - ETA: 4s - loss: 0.6576 - accuracy

186/625 [=======>......................] - ETA: 46s - loss: 0.7809 - accuracy: 0.906 - ETA: 52s - loss: 0.8203 - accuracy: 0.898 - ETA: 55s - loss: 0.7525 - accuracy: 0.895 - ETA: 56s - loss: 0.9859 - accuracy: 0.867 - ETA: 1:01 - loss: 1.0481 - accuracy: 0.86 - ETA: 1:00 - loss: 0.9680 - accuracy: 0.87 - ETA: 1:01 - loss: 0.8356 - accuracy: 0.89 - ETA: 1:01 - loss: 0.9659 - accuracy: 0.87 - ETA: 1:00 - loss: 0.8900 - accuracy: 0.88 - ETA: 1:00 - loss: 0.8589 - accuracy: 0.89 - ETA: 59s - loss: 0.8481 - accuracy: 0.8977 - ETA: 58s - loss: 0.8466 - accuracy: 0.898 - ETA: 57s - loss: 0.8101 - accuracy: 0.899 - ETA: 56s - loss: 0.8022 - accuracy: 0.896 - ETA: 56s - loss: 0.7709 - accuracy: 0.900 - ETA: 55s - loss: 0.7440 - accuracy: 0.902 - ETA: 55s - loss: 0.7627 - accuracy: 0.899 - ETA: 55s - loss: 0.7758 - accuracy: 0.901 - ETA: 55s - loss: 0.7880 - accuracy: 0.901 - ETA: 54s - loss: 0.7637 - accuracy: 0.904 - ETA: 54s - loss: 0.7293 - accuracy: 0.909 - ETA: 54s - loss: 0.7322 - accura

372/625 [================>.............] - ETA: 36s - loss: 0.5935 - accuracy: 0.920 - ETA: 36s - loss: 0.5932 - accuracy: 0.921 - ETA: 36s - loss: 0.5928 - accuracy: 0.920 - ETA: 36s - loss: 0.5928 - accuracy: 0.920 - ETA: 36s - loss: 0.5917 - accuracy: 0.920 - ETA: 35s - loss: 0.5916 - accuracy: 0.920 - ETA: 35s - loss: 0.5907 - accuracy: 0.920 - ETA: 35s - loss: 0.5929 - accuracy: 0.920 - ETA: 35s - loss: 0.5912 - accuracy: 0.920 - ETA: 35s - loss: 0.5887 - accuracy: 0.920 - ETA: 35s - loss: 0.5884 - accuracy: 0.921 - ETA: 35s - loss: 0.5912 - accuracy: 0.920 - ETA: 35s - loss: 0.5896 - accuracy: 0.921 - ETA: 35s - loss: 0.5896 - accuracy: 0.921 - ETA: 35s - loss: 0.5905 - accuracy: 0.921 - ETA: 35s - loss: 0.5905 - accuracy: 0.921 - ETA: 35s - loss: 0.5951 - accuracy: 0.920 - ETA: 35s - loss: 0.5974 - accuracy: 0.920 - ETA: 34s - loss: 0.5946 - accuracy: 0.921 - ETA: 34s - loss: 0.5920 - accuracy: 0.921 - ETA: 34s - loss: 0.5905 - accuracy: 0.921 - ETA: 34s - loss: 0.5913 - accurac

560/625 [=========================>....] - ETA: 20s - loss: 0.5937 - accuracy: 0.921 - ETA: 20s - loss: 0.5937 - accuracy: 0.921 - ETA: 20s - loss: 0.5930 - accuracy: 0.921 - ETA: 20s - loss: 0.5930 - accuracy: 0.921 - ETA: 20s - loss: 0.5929 - accuracy: 0.921 - ETA: 20s - loss: 0.5928 - accuracy: 0.921 - ETA: 20s - loss: 0.5935 - accuracy: 0.921 - ETA: 20s - loss: 0.5953 - accuracy: 0.921 - ETA: 20s - loss: 0.5971 - accuracy: 0.921 - ETA: 20s - loss: 0.5984 - accuracy: 0.921 - ETA: 20s - loss: 0.5983 - accuracy: 0.921 - ETA: 20s - loss: 0.5993 - accuracy: 0.921 - ETA: 19s - loss: 0.5991 - accuracy: 0.921 - ETA: 19s - loss: 0.5990 - accuracy: 0.921 - ETA: 19s - loss: 0.5982 - accuracy: 0.921 - ETA: 19s - loss: 0.5976 - accuracy: 0.921 - ETA: 19s - loss: 0.5965 - accuracy: 0.921 - ETA: 19s - loss: 0.5983 - accuracy: 0.921 - ETA: 19s - loss: 0.5982 - accuracy: 0.921 - ETA: 19s - loss: 0.5993 - accuracy: 0.921 - ETA: 19s - loss: 0.5987 - accuracy: 0.921 - ETA: 19s - loss: 0.5983 - accurac

625/625 [==============================] - ETA: 5s - loss: 0.5985 - accuracy: 0.92 - ETA: 5s - loss: 0.5999 - accuracy: 0.92 - ETA: 5s - loss: 0.5998 - accuracy: 0.92 - ETA: 5s - loss: 0.5989 - accuracy: 0.92 - ETA: 5s - loss: 0.5983 - accuracy: 0.92 - ETA: 4s - loss: 0.5977 - accuracy: 0.92 - ETA: 4s - loss: 0.5967 - accuracy: 0.92 - ETA: 4s - loss: 0.5958 - accuracy: 0.92 - ETA: 4s - loss: 0.5953 - accuracy: 0.92 - ETA: 4s - loss: 0.5969 - accuracy: 0.92 - ETA: 4s - loss: 0.5959 - accuracy: 0.92 - ETA: 4s - loss: 0.5959 - accuracy: 0.92 - ETA: 4s - loss: 0.5950 - accuracy: 0.92 - ETA: 4s - loss: 0.5950 - accuracy: 0.92 - ETA: 4s - loss: 0.5946 - accuracy: 0.92 - ETA: 4s - loss: 0.5936 - accuracy: 0.92 - ETA: 4s - loss: 0.5926 - accuracy: 0.92 - ETA: 3s - loss: 0.5921 - accuracy: 0.92 - ETA: 3s - loss: 0.5939 - accuracy: 0.92 - ETA: 3s - loss: 0.5948 - accuracy: 0.92 - ETA: 3s - loss: 0.5952 - accuracy: 0.92 - ETA: 3s - loss: 0.5948 - accuracy: 0.92 - ETA: 3s - loss: 0.5948 - accuracy